# Prédire l'énergie atomique à partir de la masse de la molécule

Les données sont disponibles au chemin /MolNRJPred/data/atomes/train

In [8]:
# Masses atomiques

atomic_masses = {
    "H": 1.00784,
    "He": 4.002602,
    "C": 12.00000,
    "N": 14.00674,
    "O": 15.999,
    "F": 18.998403,
    "Na": 22.989769,
    "Mg": 24.305,
    "P": 30.973762,
    "S": 32.06,
    "Cl": 35.45,
    "K": 39.0983,
    "Ca": 40.078,
    "Sc": 44.955908,
}

In [9]:
# !pip install ase
from ase.io import read
import os
import pandas as pd

In [10]:
# Pré-traitement des données

def xyz_to_mass(xyz_file):
    """
    Donne l'identifiant de la molécule et sa masse atomique
    """
    #récupère l'identifiant situé après le premier '_' et avant le '.xyz'
    id = xyz_file.split('/')[-1].split('_')[1].split('.')[0]
    atoms = read(xyz_file)
    mass = sum(atomic_masses[atom.symbol] for atom in atoms)
    return id, mass

def csv_mass(path, out_path, name, test=False):
    """
    Donne un fichier csv avec tous les identifiants de molécules et leurs masses atomiques
    """
    files = [f for f in os.listdir(path) if f.endswith('.xyz')]
    data = [xyz_to_mass(os.path.join(path, f)) for f in files]
    df = pd.DataFrame(data, columns=['id', 'mass'])
    # Ajouter les énergies depuis le fichier train.csv au chemin '/home/n7student/5ModIA/MolNRJPred/data/energies/train.csv'
    if not test:
        energies = pd.read_csv('/home/n7student/5ModIA/MolNRJPred/data/energies/train.csv')
        df['id'] = df['id'].astype(int)
        energies['id'] = energies['id'].astype(int)
        df = df.merge(energies, on='id', how='left')
    df.to_csv(os.path.join(out_path, name), index=False)
    return df

path_to_test = '../data/atoms/test'
path_to_train = '../data/atoms/train'
csv_mass(path_to_test, '.', 'masses_test.csv', test=True)
csv_mass(path_to_train, '.', 'masses_train.csv')


,id,mass,energy
0,438,86.07740,-74.930323
1,4184,93.06162,-70.779999
2,3188,83.07730,-69.265664
3,2474,98.10976,-97.212364
4,5056,97.09298,-83.956232
...,...,...,...
6586,5920,95.07730,-77.080444
6587,5519,98.09188,-76.820903
6588,4760,96.09408,-90.847735
6589,3700,88.06072,-66.890048


In [12]:
# Régression linéaire pour prédire l'énergie atomique à partir de la masse atomique
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

def train_regression_model(csv_test, csv_train):
    """
    Entraîne un modèle de régression linéaire pour prédire l'énergie atomique à partir de la masse atomique
    """
    # Charger les données
    df_test = pd.read_csv(csv_test)
    df_train = pd.read_csv(csv_train)
    
    # Séparer les caractéristiques et la cible
    X_train = df_train[['mass']]
    y_train = df_train['energy']
    X_test = df_test[['mass']]

    # Modèle de régression linéaire
    model = LinearRegression()
    model.fit(X_train, y_train)
    # Prédictions sur les données de test
    df_test['energy'] = model.predict(X_test)
    # Print RMSE de train
    from sklearn.metrics import mean_squared_error
    import numpy as np
    y_train_pred = model.predict(X_train)
    rmse_train = np.sqrt(mean_squared_error(y_train, y_train_pred))
    print(f"RMSE on training data: {rmse_train:.4f}")
    # Enlever la colonne 'mass'
    df_test = df_test.drop(columns=['mass'])
    # Sauvegarder les résultats
    df_test.to_csv('../results/predicted_energies_test.csv', index=False)
    
    return model

# Entraîner le modèle de régression
model = train_regression_model('masses_test.csv', 'masses_train.csv')

RMSE on training data: 7.0585
